# Import Library & Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from func import outlier_counter, get_all_univariate_outlier_index
from modelling_purpose import Xy, algorithm_report_accumulation

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('csv/imputed.csv')

# With / Without Outlier

Kita akan membuat 2 dataset dari `imputed.csv`, yaitu:
- Dengan Outlier
- Tanpa Outlier (Univariate)

In [3]:
df_without_outlier = df.copy()
outlier_columns  = ['TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole',
                    'YearsSinceLastPromotion', 'YearsWithCurrManager', 'TrainingTimesLastYear',
                    'NumCompaniesWorked', 'MonthlyIncome']
outlier_index = get_all_univariate_outlier_index(df_without_outlier, outlier_columns)
df_without_outlier.drop(df_without_outlier.index[outlier_index], inplace=True)

In [4]:
df.shape

(1029, 31)

In [5]:
df_without_outlier.shape

(650, 31)

`df` adalah dataset dengan outlier. Dan `df_without_outlier` adalah dataset tanpa outlier.

# Feature Engineering

In [6]:
def one_hot(df,column):
    df = pd.concat(
    [
        df,
        pd.get_dummies(df[column], prefix=column, drop_first=True)
    ],
    axis=1)
    df = df.drop(columns=column)

In [7]:
# Ini hanya untuk eksplanasi ---------------
X = df.drop('Attrition',axis=1)
y = df['Attrition'].map({'Yes':1,'No':0})

categorical_features = X.select_dtypes(include='O').columns.tolist()
ordinal = ['Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
           'JobSatisfaction', 'PerformanceRating', 'RelationshipSatisfaction',
           'StockOptionLevel', 'WorkLifeBalance']
categorical_features += ordinal
categorical_features
# Ini hanya untuk eksplanasi ---------------

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime',
 'Education',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'WorkLifeBalance']

Semua categorical features, entah nominal ataupun ordinal akan digabung dan diaplikasikan dengan One-Hot Encoder.
Alasan mengapa ordinal features juga menggunakan One-Hot Encoder adalah karena pada kasus classifier, ordinal feature yang mempunyai continuous behaviour tidak berpengaruh seperti pada kasus regressor. Melakukan pe-ranking-an pada suatu feature menjadi tidak berguna.

# Model

Pada classifier problem, kita akan memilih salah satu metrik penilaian yang akan dijadikan acuan. Ini dikarenakan False Positive dan False Negative akan selalu trade-off satu sama lain. Jadinya, umumnya kita akan dihadapkan dengan 2 pilihan berikut:
- Kasus False Negative lebih beresiko daripada kasus False Positive
- Kasus False Positive lebih beresiko daripada kasus False Negative

Di kasus ini, False Positive dan False Negative bisa diterjemahkan seperti ini:
- FP : Pegawai yang tidak keluar, terprediksi keluar.
- FN : Pegawai yang keluar, terprediksi tidak keluar.

Untuk kasus Attrition, saya menganggap kasus **False Negative adalah yang beresiko**.

Alasannya adalah, apabila ada pegawai keluar namun terprediksi tidak keluar, perusahaan beresiko kehilangan pegawai potensialnya. Dengan decision seperti ini, maka saya putuskan untuk memilih **Recall** sebagai metric yang diutamakan.

In [8]:
X, y = Xy(df)
X_wo, y_wo = Xy(df_without_outlier)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [10]:
algorithm_list = [LogisticRegression,DecisionTreeClassifier,RandomForestClassifier, GradientBoostingClassifier]

a = algorithm_report_accumulation(algorithm_list, X, y, .2, 'with Outliers')
b = algorithm_report_accumulation(algorithm_list, X_wo, y_wo, .2, 'without Outliers')
x = pd.concat([a, b],ignore_index=True)
x.set_index('Algorithm')

,Train Recall,Test Recall,Notes
Algorithm,,,
<class 'sklearn.linear_model._logistic.LogisticRegression'>,0.007092,0.028571,with Outliers
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,1.000000,0.285714,with Outliers
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,1.000000,0.142857,with Outliers
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.765957,0.228571,with Outliers
<class 'sklearn.linear_model._logistic.LogisticRegression'>,0.033333,0.136364,without Outliers
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,1.000000,0.545455,without Outliers
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,1.000000,0.272727,without Outliers
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.922222,0.318182,without Outliers


**Temuan** :
- LogisticRegression dengan default parameter underfit dengan parah. Entah di dataset dengan dan tanpa outlier, recall score tidak sampai 0.15.
- Model lainnya (selain LogisticRegression) overfit.

Dengan temuan ini, saya tidak melanjutkan untuk menggunakan LogisticRegression. Dengan asumsi bahwa dengan score serendah itu, tentunya akan memerlukan effort lebih untuk menaikkan scorenya, meskipun menggunakan Hyperparameter Tuning. Karena pada umumnya lebih mudah untuk melakukan tuning pada model yang overfit daripada yang underfit.

# Dataset Cross Validation Checking

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
def find_CVS(features,target,model):
    X_train, X_test, y_train, y_test = train_test_split(features,target,random_state=11111992)
    classification = model()
    score = cross_val_score(classification,features, target, cv=5).mean()
    return score

In [13]:
def cv_score_accumulation(algorithm_list, X, y, notes):
    cv_score = []
    notes_arr = []
    for i in algorithm_list :
        score = find_CVS(X,y,i)
        cv_score.append(score)
        notes_arr.append(notes)

    cv_df = pd.DataFrame({
        'Algorithm': algorithm_list,
        'Notes': notes_arr,
        'CV Score': cv_score
    })
    return cv_df

In [14]:
algorithm_list = [DecisionTreeClassifier,RandomForestClassifier, GradientBoostingClassifier]
a = cv_score_accumulation(algorithm_list, X, y, 'with Outliers')
b = cv_score_accumulation(algorithm_list, X_wo, y_wo, 'without Outliers')
x = pd.concat([a, b],ignore_index=True)
x.set_index('Algorithm')

,Notes,CV Score
Algorithm,,
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,with Outliers,0.779384
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,with Outliers,0.848392
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,with Outliers,0.854246
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,without Outliers,0.763077
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,without Outliers,0.847692
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,without Outliers,0.846154


**Temuan** : Dengan data ini ditemukan bahwa CV Score dari dataset dengan Outliers lebih baik daripada CV Score dari datest tanpa Outliers.

Maka, selanjutnya kita akan menggunakan algoritma DecisionTree, RandomForest, juga GradientBoosting menggunakan dataset dengan outliers untuk Hyperparameter Tuning.

# Hyperparameter Tuning